# Defining materials

When performing chemical equilibrium or chemical kinetics calculations, an **initial chemical state** is required. In most cases, you should be able to construct an initial state in terms of *given amounts of chemical species in the system* using the [ChemicalState](https://reaktoro.org/api/classReaktoro_1_1ChemicalState.html) class. However, this may not always be convenient or possible for the user. The [Material](https://reaktoro.org/api/classReaktoro_1_1Material.html) class offers an alternative way to define an initial chemical state.

## An initial chemical state not so conveniently defined

Consider an **aqueous solution** given by the following *recipe*: 
* 1 molal NaCl, 
* 0.1 molal CaCl<sub>2</sub>, 
* 0.05 molal MgCl<sub>2</sub> and 
* 0.1 molal CO<sub>2</sub>. 

Consider a **rock** with the following mass composition: 
* 80%<sub>kg</sub> quartz (SiO<sub>2</sub>) and 
* 20%<sub>kg</sub> calcite (CaCO<sub>3</sub>). 

Mix 1 kg of this *aqueous solution* with 10 g of *rock* and find the chemical equilibrium state of the system at 60°C and 15 bar.

We start with the definition of a suitable chemical system to model this problem:

In [ ]:
from reaktoro import *

db = PhreeqcDatabase("phreeqc.dat")

solution = AqueousPhase()
solution.setActivityModel(ActivityModelPitzerHMW())

minerals = MineralPhases("Halite Calcite Dolomite Quartz")

system = ChemicalSystem(db, solution, minerals)

**TASK 1**: inspect, which aqueous species were selected automatically based on the chemical elements composing the explicitly listed minerals.

In [ ]:
# -------------------------------------- #
# Code cell for the task
# -------------------------------------- #

Next, we create an initial chemical state containing 1 kg of aqueous solution and 10 g of rock:

In [ ]:
state = ChemicalState(system)

# Set initial brine composition
state.set("H2O" , 1.00, "kg")
state.set("Na+" , 1.00, "mol")
state.set("Ca+2", 0.10, "mol")
state.set("Mg+2", 0.05, "mol")
state.set("Cl-" , 1.30, "mol") # = 1.0 + 2*0.1 + 2*0.05
state.set("CO2" , 0.10, "mol")

# Set initial rock composition
state.set("Quartz",  80.0, "g")
state.set("Calcite", 20.0, "g")

# Scale fluid and solid masses to 1 kg and 10 g, respectively
state.scaleFluidMass(1.0, "kg")
state.scaleSolidMass(10.0, "g")

While defining this initial chemical state, we need:

* to specify amounts for ions `Na+`, `Ca+2`, and `Mg+2` that reflect the desired molality of NaCl, CaCl<sub>2</sub>, and MgCl<sub>2</sub>,
* to specify an amount for `Cl-` that produces zero electric charge in the solution,
* to provide species names exactly how they exist in the database instead of chemical formulas (e.g., names `Quartz` and `Calcite` instead of formulas `SiO2` and `CaCO3`), and
* to scale the mass of fluid and solids in the chemical state.

Let's now equilibrate this state at 60 °C and 15 bar:

In [ ]:
state.temperature(60.0, "celsius")
state.pressure(15.0, "bar")

equilibrate(state)

print(state)

## Simplifying with class Material

The same result can be obtained with the class [Material](https://reaktoro.org/api/classReaktoro_1_1Material.html):

In [ ]:
brine = Material(system)
brine.add("H2O"  , 1.00, "kg")
brine.add("NaCl" , 1.00, "mol")
brine.add("CaCl2", 0.10, "mol")
brine.add("MgCl2", 0.05, "mol")
brine.add("CO2"  , 0.10, "mol")

rock = Material(system)
rock.add("SiO2",  80.0, "g")
rock.add("CaCO3", 20.0, "g")

mix = brine(1.0, "kg") + rock(10.0, "g")

state = mix.equilibrate(60, "celsius", 15, "bar")

print(state)

**TASK 2**: check if the obtained states are indeed the same.